# 미세먼지 , 한파, 폭염 상관분석

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 미세먼지 데이터 저장
fine_dust_ds_1 = pd.read_csv('./data/미세먼지/서울시 대기질 자료 제공_2020-2021.csv', encoding=('euc-kr'))
fine_dust_ds_1 = fine_dust_ds_1.rename(columns={fine_dust_ds_1.columns[-1] : '초미세먼지(PM2.5)'})
fine_dust_ds_2 = pd.read_csv('./data/미세먼지/서울시 대기질 자료 제공_2022.csv', encoding=('euc-kr'))
fine_dust_ds_2.rename(columns={fine_dust_ds_2.columns[-1] : '초미세먼지(PM2.5)'})

fine_dust_ds = pd.concat([fine_dust_ds_1, fine_dust_ds_2], axis=0)
fine_dust_ds


,일시,구분,미세먼지(PM10),초미세먼지(PM2.5)
0,2021-12-31 23:00,평균,21.0,9.0
1,2021-12-31 23:00,강남구,21.0,9.0
2,2021-12-31 23:00,강동구,25.0,7.0
3,2021-12-31 23:00,강북구,23.0,12.0
4,2021-12-31 23:00,강서구,28.0,9.0
...,...,...,...,...
456137,2020-01-01 0:00,용산구,12.0,13.0
456138,2020-01-01 0:00,은평구,18.0,12.0
456139,2020-01-01 0:00,종로구,20.0,15.0
456140,2020-01-01 0:00,중구,22.0,18.0


,일시,구분,미세먼지(PM10),초미세먼지(PM2.5)
0,2022-12-31 23:00,평균,59.0,46.0
1,2022-12-31 23:00,강남구,57.0,44.0
2,2022-12-31 23:00,강동구,68.0,55.0
3,2022-12-31 23:00,강북구,59.0,42.0
4,2022-12-31 23:00,강서구,62.0,40.0
...,...,...,...,...
227755,2022-01-01 0:00,용산구,22.0,11.0
227756,2022-01-01 0:00,은평구,19.0,9.0
227757,2022-01-01 0:00,종로구,20.0,9.0
227758,2022-01-01 0:00,중구,20.0,11.0


,일시,구분,미세먼지(PM10),초미세먼지(PM2.5)
0,2021-12-31 23:00,평균,21.0,9.0
1,2021-12-31 23:00,강남구,21.0,9.0
2,2021-12-31 23:00,강동구,25.0,7.0
3,2021-12-31 23:00,강북구,23.0,12.0
4,2021-12-31 23:00,강서구,28.0,9.0
...,...,...,...,...
227755,2022-01-01 0:00,용산구,22.0,11.0
227756,2022-01-01 0:00,은평구,19.0,9.0
227757,2022-01-01 0:00,종로구,20.0,9.0
227758,2022-01-01 0:00,중구,20.0,11.0


In [10]:
# 폭염 데이터 저장
# 일시 | 지점 | 최고체감온도 | 최고기온 | 평균기온 | 최저기온 | 평균상대습도
heatwave_ds = pd.read_csv('./data/폭염/S-DoT_NATURE_2020.05.01-05.31.csv', encoding='euc-kr')
heatwave_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596979 entries, 0 to 596978
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   기관 명          596979 non-null  object 
 1   모델명           596979 non-null  object 
 2   시리얼           596979 non-null  object 
 3   구분            596979 non-null  int64  
 4    기온(℃)        596958 non-null  float64
 5   상대습도( %)      596958 non-null  float64
 6   풍향(°)         7205 non-null    float64
 7   풍속(m/s)       7205 non-null    float64
 8   돌풍 풍향(°)      7205 non-null    float64
 9   돌풍 풍속(m/s)    7205 non-null    float64
 10  조도(lux)       594226 non-null  float64
 11  자외선(UVI)      506462 non-null  float64
 12  소음(dB)        594621 non-null  float64
 13   진동_x(g)      594620 non-null  float64
 14   진동_y(g)      594620 non-null  float64
 15   진동_z(g)      594620 non-null  float64
 16   진동_x 최대(g)   594620 non-null  float64
 17   진동_y 최대(g)   594620 non-null  float64
 18   진동_

In [ ]:
# 목적에 맞춰서 결측치를 제거 -> 난 제거할듯
# fine_dust_ds = fine_dust_ds.dropna(axis=0)
# fine_dust_ds = fine_dust_ds.dropna(axis=1, thresh=len(fine_dust_ds)*0.7)
# 일단 초미세먼지 
fine_dust_ds = fine_dust_ds.dropna(subset='미세먼지(PM10)', axis=0)
print(fine_dust_ds.isna().sum())
fine_dust_ds

일시                   0
구분                   0
미세먼지(PM10)           0
초미세먼지(PM25)     225011
초미세먼지(PM2.5)    447657
dtype: int64


,일시,구분,미세먼지(PM10),초미세먼지(PM25),초미세먼지(PM2.5)
0,2021-12-31 23:00,평균,21.0,9.0,NaN
1,2021-12-31 23:00,강남구,21.0,9.0,NaN
2,2021-12-31 23:00,강동구,25.0,7.0,NaN
3,2021-12-31 23:00,강북구,23.0,12.0,NaN
4,2021-12-31 23:00,강서구,28.0,9.0,NaN
...,...,...,...,...,...
227755,2022-01-01 0:00,용산구,22.0,NaN,11.0
227756,2022-01-01 0:00,은평구,19.0,NaN,9.0
227757,2022-01-01 0:00,종로구,20.0,NaN,9.0
227758,2022-01-01 0:00,중구,20.0,NaN,11.0


In [12]:
# 시리얼 매핑 데이터 
def extract_gu_split(address):
    if pd.isna(address):
        return None
    parts = str(address).split()
    for i, part in enumerate(parts):
        if part.endswith('구') and len(part) >= 2 and len(part) <= 4:
            return part
    return None

serial_ds = pd.read_excel('./data/서울시 도시데이터 센서(S-DoT) 환경정보 설치 위치정보.xlsx')

# 모델 시리얼 컬럼명 변경 + 앞뒤 공백 완전 제거
serial_ds = serial_ds.rename(columns={'모델 시리얼(*)': '시리얼'})
serial_ds['시리얼'] = serial_ds['시리얼'].astype(str).str.strip().str.replace(r'^\s+', '', regex=True)

# 
serial_ds['주소'] = serial_ds['주소'].apply(extract_gu_split)

# 3) 필요한 컬럼만 선택 (주소 대신 구 사용)
serial_ds = serial_ds[['시리얼', '주소']]

display(serial_ds.head())
print("시리얼 공백 제거 후:")
print(serial_ds['시리얼'].str.len().value_counts())  # 길이 확인


,시리얼,주소
0,V02Q1940059,강남구
1,V02Q1940293,강남구
2,V02Q1940060,강남구
3,V02Q1940079,강남구
4,V02Q1940129,강남구


시리얼 공백 제거 후:
시리얼
11    1136
12      23
Name: count, dtype: int64


In [13]:
data_dir = "./data/폭염"
excel_files = glob.glob(os.path.join(data_dir, "S-DoT_NATURE_*.csv"))

all_avgs = []  # 리스트로 누적

for path in excel_files:
    print(f"처리 중: {os.path.basename(path)}")
    
    heatwave_ds = pd.read_csv(path, encoding=('euc-kr'))  # 파일 로드
    heatwave_ds.columns = ['기관명', '모델명', '시리얼', '구분', '기온(℃)', '상대습도(%)', 
                      '풍향(°)', '풍속(m/s)', '돌풍 풍향(°)', '돌풍 풍속(m/s)', 
                      '조도(lux)', '자유선 (UVI)', '소음 (dB)', '진동(x) (g)', 
                      '진동(y)(g)', '진동(z) (g)', '진동(x)최대(g)', '진동(y) 최대(g)', 
                      '진동(z) 최대(g)', '흑구 온도(℃)', '측정시간', '등록일자']
    # 등록일자 처리
    # heatwave_ds['등록일자'] = pd.to_datetime(heatwave_ds['등록일자'], format='%Y.%m.%d %H:%M')
    
    # heatwave_ds['등록일자'] = pd.to_datetime(heatwave_ds['등록일자'], format='%Y.%m.%d %H:%M')
    
    heatwave_ds['등록일자'] = pd.to_datetime(heatwave_ds['등록일자'], errors='coerce')  # 자동
    heatwave_ds = heatwave_ds.dropna(subset=['등록일자', '기온(℃)'])

    heatwave_ds['등록일자'] = heatwave_ds['등록일자'].dt.floor('H')
    # merge (시리얼 기준)
    heatwave_ds = heatwave_ds.merge(
        serial_ds[['시리얼', '주소']], 
        on='시리얼', 
        how='left'
    )
    
    # 주소 없는 행 삭제
    heatwave_ds = heatwave_ds.dropna(subset=['주소'])
    
     # 그룹 평균 계산
    file_avg_heat = heatwave_ds.groupby(['등록일자', '주소'])['기온(℃)'].mean().reset_index()
    file_avg_heat.columns = ['등록일자', '주소', '기온_평균']
    
    all_avgs.append(file_avg_heat)
    print(f"  → {len(file_avg_heat)}행 추가")

처리 중: S-DoT_NATURE_2020.05.01-05.31.csv
  → 17775행 추가
처리 중: S-DoT_NATURE_2020.06.01-06.30.csv
  → 12550행 추가
처리 중: S-DoT_NATURE_2020.07.01-07.31.csv
  → 17775행 추가
처리 중: S-DoT_NATURE_2020.08.01-08.31.csv
  → 17825행 추가
처리 중: S-DoT_NATURE_2020.09.01-09.30.csv
  → 18000행 추가
처리 중: S-DoT_NATURE_2021.05.03-05.09.csv
  → 2760행 추가
처리 중: S-DoT_NATURE_2021.05.17-05.23.csv
  → 4170행 추가
처리 중: S-DoT_NATURE_2021.05.24-05.30.csv
  → 4146행 추가
처리 중: S-DoT_NATURE_2021.05.31-06.06.csv
  → 4013행 추가
처리 중: S-DoT_NATURE_2021.06.07-06.13.csv
  → 3817행 추가
처리 중: S-DoT_NATURE_2021.06.14-06.20.csv
  → 4178행 추가
처리 중: S-DoT_NATURE_2021.06.21-06.27.csv
  → 4175행 추가
처리 중: S-DoT_NATURE_2021.06.28-07.04.csv
  → 4200행 추가
처리 중: S-DoT_NATURE_2021.07.05-07.11.csv
  → 4200행 추가
처리 중: S-DoT_NATURE_2021.07.12-07.18.csv
  → 4182행 추가
처리 중: S-DoT_NATURE_2021.07.19-07.25.csv
  → 4164행 추가
처리 중: S-DoT_NATURE_2021.07.26-08.01.csv
  → 4200행 추가
처리 중: S-DoT_NATURE_2021.08.02-08.08.csv
  → 4200행 추가
처리 중: S-DoT_NATURE_2021.08.09-08.15.csv
 

In [14]:
# 2. 모든 파일 결과 concat
final_avg_temp_heat = pd.concat(all_avgs)

# 중복 제거 (같은 날짜+주소만 남김)
final_avg_temp_heat = final_avg_temp_heat.groupby(['등록일자', '주소'])['기온_평균'].mean().reset_index()
final_avg_temp_heat

,등록일자,주소,기온_평균
0,2020-05-01 01:00:00,강남구,19.423529
1,2020-05-01 01:00:00,강동구,19.872414
2,2020-05-01 01:00:00,강북구,19.260870
3,2020-05-01 01:00:00,강서구,18.903333
4,2020-05-01 01:00:00,관악구,19.060870
...,...,...,...
246208,2022-10-02 23:00:00,용산구,18.364000
246209,2022-10-02 23:00:00,은평구,16.075000
246210,2022-10-02 23:00:00,종로구,17.565385
246211,2022-10-02 23:00:00,중구,17.763158


In [15]:
data_dir = "./data/한파"
excel_files = glob.glob(os.path.join(data_dir, "S-DoT_NATURE_*.csv"))

all_avgs = []  # 리스트로 누적

for path in excel_files:
    print(f"처리 중: {os.path.basename(path)}")
    
    coldwave_ds = pd.read_csv(path, encoding=('euc-kr'))  # 파일 로드
    coldwave_ds = coldwave_ds.rename(columns={
    coldwave_ds.columns[2]: '시리얼',
    coldwave_ds.columns[4]: '기온(℃)',
    coldwave_ds.columns[-1]: '등록일자'
})
    coldwave_ds['기온(℃)'] = pd.to_numeric(coldwave_ds['기온(℃)'], errors='coerce')
    coldwave_ds['시리얼'] = coldwave_ds['시리얼'].astype(str).str.strip()
    # 등록일자 처리
    coldwave_ds['등록일자'] = pd.to_datetime(coldwave_ds['등록일자'], errors='coerce')  # 자동
    coldwave_ds = coldwave_ds.dropna(subset=['등록일자', '기온(℃)'])

    coldwave_ds['등록일자'] = coldwave_ds['등록일자'].dt.floor('H')
    # merge (시리얼 기준)
    coldwave_ds = coldwave_ds.merge(
        serial_ds[['시리얼', '주소']], 
        on='시리얼', 
        how='left'
    )
    
    # 주소 없는 행 삭제
    coldwave_ds = coldwave_ds.dropna(subset=['주소'])
    
     # 그룹 평균 계산
    file_avg_cold = coldwave_ds.groupby(['등록일자', '주소'])['기온(℃)'].mean().reset_index()
    file_avg_cold.columns = ['등록일자', '주소', '기온_평균']
    
    all_avgs.append(file_avg_cold)
    print(f"  → {len(file_avg_cold)}행 추가")

처리 중: S-DoT_NATURE_2020.10.01-10.31.csv
  → 18600행 추가
처리 중: S-DoT_NATURE_2020.11.09-11.15.csv
  → 2664행 추가
처리 중: S-DoT_NATURE_2020.11.16-11.22.csv
  → 1450행 추가
처리 중: S-DoT_NATURE_2020.11.23-11.29.csv
  → 2765행 추가
처리 중: S-DoT_NATURE_2020.11.30-12.06.csv
  → 3100행 추가
처리 중: S-DoT_NATURE_2020.12.07-12.13.csv
  → 3233행 추가
처리 중: S-DoT_NATURE_2020.12.14-12.20.csv
  → 857행 추가
처리 중: S-DoT_NATURE_2020.12.21-12.27.csv
  → 3949행 추가
처리 중: S-DoT_NATURE_2020.12.28-01.03.csv
  → 1450행 추가
처리 중: S-DoT_NATURE_2021.01.04-01.10.csv
  → 1248행 추가
처리 중: S-DoT_NATURE_2021.01.11-01.17.csv
  → 3239행 추가
처리 중: S-DoT_NATURE_2021.01.18-01.24.csv
  → 2506행 추가
처리 중: S-DoT_NATURE_2021.01.25-01.31.csv
  → 4200행 추가
처리 중: S-DoT_NATURE_2021.02.01-02.07.csv
  → 2729행 추가
처리 중: S-DoT_NATURE_2021.02.08-02.14.csv
  → 4175행 추가
처리 중: S-DoT_NATURE_2021.02.15-02.21.csv
  → 4102행 추가
처리 중: S-DoT_NATURE_2021.02.22-02.28.csv
  → 2900행 추가
처리 중: S-DoT_NATURE_2021.03.01-03.07.csv
  → 2825행 추가
처리 중: S-DoT_NATURE_2021.03.08-03.14.csv
  → 42

In [16]:
# 2. 모든 파일 결과 concat
final_avg_temp_cold = pd.concat(all_avgs)

# 중복 제거 (같은 날짜+주소만 남김)
final_avg_temp_cold = final_avg_temp_cold.groupby(['등록일자', '주소'])['기온_평균'].mean().reset_index()
final_avg_temp_cold

,등록일자,주소,기온_평균
0,2020-10-01 00:00:00,강남구,19.491176
1,2020-10-01 00:00:00,강동구,19.903448
2,2020-10-01 00:00:00,강북구,19.457143
3,2020-10-01 00:00:00,강서구,19.168966
4,2020-10-01 00:00:00,관악구,19.237681
...,...,...,...
255092,2022-12-28 23:00:00,용산구,-2.369231
255093,2022-12-28 23:00:00,은평구,-5.339286
255094,2022-12-28 23:00:00,종로구,-2.978431
255095,2022-12-28 23:00:00,중구,-3.566667
